
# 🧠 Receptive Field Mapping Notebook

Welcome to the **Receptive Field Mapping Notebook** — a streamlined alternative to the Streamlit app for analyzing behaviorally-relevant neural activity through high-precision video tracking and spike data alignment.

This notebook mirrors the Streamlit experience step by step while letting you stay inside Jupyter.



## 🔍 What This Notebook Does

- **Video Analysis with DeepLabCut** — Track motion of painted markers and colored filaments using your DeepLabCut project.
- **Model Re-training** — Re-train the supplied model with your own labels when you need to adapt the detector to a new setup.
- **Data Cleaning & Quality Control** — Detect outliers frame-to-frame and impute them with machine-learning models.
- **Feature Extraction & Bending Detection** — Calculate bending coefficients from tracked filament coordinates over time.
- **Spike-Time Alignment** — Synchronize neural recordings with behavioral events, including touch timestamps and filament bending.
- **Interactive Visualization** — Plot synchronized motion and spike activity, create homography visualizations, and export receptive field videos.



## ⚙️ Quick Background Summary

This workflow uses **DeepLabCut** for marker tracking, aligns **neural spikes** to tracked behavioral events, and enables visualization directly in this notebook with **Plotly** and **Matplotlib** helpers from the project.


![Backend logic flow for creating a labeled video then post-processing.](assets/flowchart.png)



## ✅ Before You Start

Make sure your recordings follow the setup instructions in the **Recording Instructions and Requirements** section below so that predictions and post-processing behave as expected.



## 🎥 Recording Instructions and Requirements

For the recording to be properly recognized by the AI model, a few things need to be marked correctly to allow for accurate predictions and follow-up post-processing.

### 📋 Pre-filming Requirements

The model is designed to detect:
- 4 dots on the skin that represent the 4 corners of a square with sides of **1 or 2 cm**.
- A filament with 3 **separate color zones**, allowing it to distinguish **6 points for bending**.

#### ✅ Lighting and Camera
- Use **static, clean white lighting** for the subject.
- Ensure a **stationary camera** throughout the recording. Either use a tripod or have steady arms.

#### ✅ Skin Dot Marking
- Paint **4 dots** in the corners of a square using a **bright, opaque green** marker.
  - Recommended: [Posca paint pens](https://www.posca.com/en/product/pc-5m/)
- Avoid having other objects or markings in the frame that could be confused with the dots.
- Example of bright green dots on skin, in a well-lit setting:

![Example: Bright green dots on skin](assets/dots_example.png)

#### ✅ Filament Marking
- Paint the filament with **2 distinct opaque colors in pattern**.
- This allows the model to detect **6 separate points** for bend analysis.
- Avoid similar colors or objects to the filament that could confuse the model.
- Example of a filament painted white and dark blue:

![Example: Colored filament with clear zones](assets/filament_example.png)

#### ✅ Clean the Skin
- Ensure no old marks or blemishes interfere with detection.

### 🎬 During-Filming Requirements
- Start the video with **5 touches** at the hotspot — one per second — for synchronization with neuron data.
- Ensure **no other filaments are visible** in the frame.
- Confirm that the **filament is clearly visible and not blurry** during bending.

<table>
  <tr>
    <td><img src="assets/bad_bend_example_1.png" width="220"><br>❌ Blurry region of interest</td>
    <td><img src="assets/bad_bend_example_2.png" width="220"><br>❌ Poor bend angle</td>
    <td><img src="assets/good_bend_example.png" width="220"><br>✅ Clear, visible bend</td>
  </tr>
</table>

> ✅ Double-check recordings to ensure clear visibility of the bend and proper lighting.



## 🎬 Step 1 — DeepLabCut Video Prediction

This section mirrors the Streamlit **Create Labeled Video** and **Labeling / Retraining** tabs. Use it to initialise your DeepLabCut project, preprocess a video, generate predictions, extract frames for labeling, and kick off retraining when needed.

Run the setup cell below first, then interact with the widgets to walk through the workflow.


In [ ]:
# --- Notebook helpers (run this cell once) ---
import tempfile
from pathlib import Path
from glob import glob
import json

import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, Image, Video, JSON, clear_output

import pandas as pd
import plotly.express as px
import matplotlib.cm as cm

from src.train_predict import dlc_utils
from src.post_processing.datadlc import DataDLC
from src.post_processing.dataneuron import DataNeuron
from src.post_processing.mergeddata import MergedData
from src.post_processing.outlierimputer import OutlierImputer
from src.post_processing.plotting_plotly import PlottingPlotly

NOTEBOOK_STATE: dict[str, object] = {}

class StreamlitNotebookShim:
    """Lightweight shim so Streamlit helper functions can print inside notebooks."""

    def __init__(self, state: dict[str, object]):
        self.session_state = state
        self._output: widgets.Output | None = None

    def set_output(self, output: widgets.Output | None) -> None:
        self._output = output

    def _display(self, obj) -> None:
        if self._output is not None:
            with self._output:
                display(obj)
        else:
            display(obj)

    def _display_markdown(self, text: str) -> None:
        self._display(Markdown(text))

    def success(self, text: str) -> None:
        self._display_markdown(f"✅ {text}")

    def info(self, text: str) -> None:
        self._display_markdown(f"ℹ️ {text}")

    def warning(self, text: str) -> None:
        self._display_markdown(f"⚠️ {text}")

    def error(self, text: str) -> None:
        self._display_markdown(f"❌ {text}")

    def write(self, obj) -> None:
        self._display(obj)

    def markdown(self, text: str) -> None:
        self._display_markdown(text)

    def title(self, text: str) -> None:
        self._display_markdown(f"# {text}")

    def header(self, text: str) -> None:
        self._display_markdown(f"## {text}")

    def subheader(self, text: str) -> None:
        self._display_markdown(f"### {text}")

    def image(self, image, caption: str | None = None, width: int | None = None) -> None:
        if isinstance(image, (str, Path)):
            img = Image(filename=str(image), width=width)
        else:
            img = Image(data=image, width=width)
        self._display(img)
        if caption:
            self._display_markdown(f"*{caption}*")

    def video(self, data, format: str = "mp4") -> None:
        if isinstance(data, (bytes, bytearray)):
            vid = Video(data=data, embed=True)
        else:
            vid = Video(filename=str(data))
        self._display(vid)

    def json(self, obj) -> None:
        self._display(JSON(obj))

    def plotly_chart(self, fig, use_container_width: bool = True) -> None:
        fig.show()

    def pyplot(self, fig, use_container_width: bool = True) -> None:
        self._display(fig)

    def stop(self) -> None:
        raise RuntimeError("Execution stopped by StreamlitNotebookShim.stop().")

NOTEBOOK_STREAMLIT = StreamlitNotebookShim(NOTEBOOK_STATE)
dlc_utils.st = NOTEBOOK_STREAMLIT

def get_all_plotly_cmaps() -> dict[str, list[str]]:
    cmap_dict: dict[str, list[str]] = {}
    for cmap_group in [px.colors.sequential,
                       px.colors.diverging,
                       px.colors.cyclical,
                       px.colors.qualitative]:
        for name in dir(cmap_group):
            if not name.startswith('_'):
                cmap_dict[name] = getattr(cmap_group, name)
    return cmap_dict

def get_all_matplotlib_cmaps() -> dict[str, object]:
    cmap_dict: dict[str, object] = {}
    for name in dir(cm):
        if not name.startswith('_'):
            cmap_dict[name] = getattr(cm, name)
    return cmap_dict

PLOTLY_CMAPS = get_all_plotly_cmaps()
MATPLOTLIB_CMAPS = get_all_matplotlib_cmaps()


In [ ]:
# --- DeepLabCut prediction & retraining interface ---

def build_prediction_workflow() -> widgets.Tab:
    style = {'description_width': '160px'}
    full_width = widgets.Layout(width='100%')

    # --- Tab 1: Create labeled video ---
    project_input = widgets.Text(
        description='Project path:',
        placeholder='Full path to your DeepLabCut project folder',
        style=style,
        layout=full_width
    )
    load_button = widgets.Button(description='Load project', icon='folder-open')
    load_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})

    video_upload = widgets.FileUpload(
        accept='.mp4,.avi,.mov',
        multiple=False,
        description='Upload video'
    )
    preprocess_button = widgets.Button(description='Preprocess video', icon='cogs')
    preprocess_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})

    prediction_button = widgets.Button(
        description='Run prediction & create labeled video',
        icon='film'
    )
    prediction_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})

    def on_load_clicked(_):
        with load_output:
            clear_output()
            path_text = project_input.value.strip()
            if not path_text:
                print('Enter the DeepLabCut project path to continue.')
                return
            project_path = Path(path_text.strip('"').strip("'"))
            if not project_path.exists():
                print(f'Path not found: {project_path}')
                return

            config_path = project_path / 'config.yaml'
            if not config_path.exists():
                print(f'config.yaml not found in {project_path}')
                return

            videos_dir = project_path / 'videos'
            videos_dir.mkdir(parents=True, exist_ok=True)
            train_folders = sorted(project_path.glob('dlc-models-pytorch/iteration-0/*/train'))
            train_folder = train_folders[0] if train_folders else None

            NOTEBOOK_STATE['project_path'] = project_path
            NOTEBOOK_STATE['config_path'] = config_path
            NOTEBOOK_STATE['videos_dir'] = videos_dir
            NOTEBOOK_STATE['training_folder'] = train_folder

            NOTEBOOK_STREAMLIT.set_output(load_output)
            try:
                dlc_utils.init_project(str(config_path), str(project_path))
                if train_folder:
                    dlc_utils.clean_snapshots(str(train_folder))
            finally:
                NOTEBOOK_STREAMLIT.set_output(None)

            summary = [
                f'Project loaded: {project_path}',
                f'Videos directory: {videos_dir}',
                f"Training folder: {train_folder if train_folder else 'not found (will be created after training)'}"
            ]
            for line in summary:
                print(line)

    load_button.on_click(on_load_clicked)

    def on_preprocess_clicked(_):
        with preprocess_output:
            clear_output()
            project_path = NOTEBOOK_STATE.get('project_path')
            videos_dir = NOTEBOOK_STATE.get('videos_dir')
            if project_path is None or videos_dir is None:
                print('Load a DeepLabCut project before preprocessing a video.')
                return
            if not video_upload.value:
                print('Upload a video file to preprocess.')
                return

            upload = next(iter(video_upload.value.values()))
            original_name = upload['metadata']['name']
            temp_input_path = videos_dir / original_name
            with open(temp_input_path, 'wb') as f:
                f.write(upload['content'])

            processed_video_name = f"processed_{Path(original_name).stem}.mp4"
            processed_video_path = videos_dir / processed_video_name

            print('Preprocessing video... this may take a while.')
            dlc_utils.preprocess_video(str(temp_input_path), str(processed_video_path))
            try:
                temp_input_path.unlink()
            except FileNotFoundError:
                pass

            NOTEBOOK_STATE['processed_video_path'] = processed_video_path
            NOTEBOOK_STATE['processed_video_name'] = processed_video_name
            print(f'Video preprocessed and saved to {processed_video_path}')

    preprocess_button.on_click(on_preprocess_clicked)

    def on_prediction_clicked(_):
        with prediction_output:
            clear_output()
            config_path = NOTEBOOK_STATE.get('config_path')
            processed_video_path = NOTEBOOK_STATE.get('processed_video_path')
            videos_dir = NOTEBOOK_STATE.get('videos_dir')

            if None in (config_path, processed_video_path, videos_dir):
                print('Load the project and preprocess a video before running predictions.')
                return

            NOTEBOOK_STREAMLIT.set_output(prediction_output)
            try:
                dlc_utils.predict_and_show_labeled_video(
                    str(config_path),
                    str(processed_video_path),
                    str(videos_dir)
                )
                if 'h5_path' in NOTEBOOK_STATE:
                    print(f"Latest DLC data stored at {NOTEBOOK_STATE['h5_path']}")
            except Exception as exc:
                print(f'Prediction failed: {exc}')
            finally:
                NOTEBOOK_STREAMLIT.set_output(None)

    prediction_button.on_click(on_prediction_clicked)

    tab1_box = widgets.VBox([
        widgets.HTML('<h3>Create Labeled Video</h3><p>Load a project, upload a video, preprocess it and generate predictions.</p>'),
        project_input,
        load_button,
        load_output,
        widgets.HTML('<hr>'),
        video_upload,
        preprocess_button,
        preprocess_output,
        widgets.HTML('<hr>'),
        prediction_button,
        prediction_output
    ], layout=widgets.Layout(gap='0.6em'))

    # --- Tab 2: Labeling / Retraining ---
    num_frames_slider = widgets.IntSlider(
        value=10,
        min=5,
        max=50,
        step=5,
        description='Frames to extract',
        style=style
    )
    extract_button = widgets.Button(description='Extract frames & launch Napari', icon='image')
    extract_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})

    epochs_slider = widgets.IntSlider(
        value=25,
        min=5,
        max=50,
        step=5,
        description='Pose epochs',
        style=style
    )
    detector_epochs_slider = widgets.IntSlider(
        value=50,
        min=5,
        max=100,
        step=5,
        description='Detector epochs',
        style=style
    )
    retrain_button = widgets.Button(description='Retrain model', icon='redo')
    retrain_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})

    def on_extract_clicked(_):
        with extract_output:
            clear_output()
            config_path = NOTEBOOK_STATE.get('config_path')
            processed_video_path = NOTEBOOK_STATE.get('processed_video_path')
            if None in (config_path, processed_video_path):
                print('Load a project and preprocess a video before extracting frames.')
                return
            NOTEBOOK_STREAMLIT.set_output(extract_output)
            try:
                dlc_utils.update_num_frames2pick(str(config_path), int(num_frames_slider.value))
                dlc_utils.run_labeling(str(config_path), str(processed_video_path))
            except Exception as exc:
                print(f'Frame extraction / labeling failed: {exc}')
            finally:
                NOTEBOOK_STREAMLIT.set_output(None)

    extract_button.on_click(on_extract_clicked)

    def on_retrain_clicked(_):
        with retrain_output:
            clear_output()
            project_path = NOTEBOOK_STATE.get('project_path')
            config_path = NOTEBOOK_STATE.get('config_path')
            processed_video_path = NOTEBOOK_STATE.get('processed_video_path')
            videos_dir = NOTEBOOK_STATE.get('videos_dir')
            train_folder = NOTEBOOK_STATE.get('training_folder')

            if None in (project_path, config_path, processed_video_path, videos_dir):
                print('Load project, preprocess a video, and extract frames before retraining.')
                return

            if train_folder is None or not Path(train_folder).exists():
                train_folders = sorted(Path(project_path).glob('dlc-models-pytorch/iteration-0/*/train'))
                train_folder = train_folders[0] if train_folders else None
                NOTEBOOK_STATE['training_folder'] = train_folder

            if train_folder is None:
                print('Could not find a training folder. Run DeepLabCut training at least once to create it.')
                return

            if not dlc_utils.is_labeling_done(str(project_path)):
                print('No labeled data found. Label and save frames in Napari before retraining.')
                return

            NOTEBOOK_STREAMLIT.set_output(retrain_output)
            try:
                dlc_utils.add_video_to_config(str(config_path), str(processed_video_path))
                dlc_utils.clean_snapshots(str(train_folder))
                dlc_utils.delete_prev_pred(str(videos_dir))
                dlc_utils.clear_training_datasets(str(project_path))

                dlc_utils.run_retraining(
                    str(config_path),
                    str(train_folder),
                    int(epochs_slider.value),
                    int(detector_epochs_slider.value)
                )

                pose_fig = dlc_utils.show_pose_training_loss(str(train_folder))
                if pose_fig:
                    display(pose_fig)
                detector_fig = dlc_utils.show_detector_training_loss(str(train_folder))
                if detector_fig:
                    display(detector_fig)

                dlc_utils.predict_and_show_labeled_video(
                    str(config_path),
                    str(processed_video_path),
                    str(videos_dir)
                )
            except Exception as exc:
                print(f'Retraining failed: {exc}')
            finally:
                NOTEBOOK_STREAMLIT.set_output(None)

    retrain_button.on_click(on_retrain_clicked)

    tab2_box = widgets.VBox([
        widgets.HTML('<h3>Labeling / Retraining</h3><p>Extract frames for Napari labeling, then retrain the model once labels are saved.</p>'),
        num_frames_slider,
        extract_button,
        extract_output,
        widgets.HTML('<hr>'),
        epochs_slider,
        detector_epochs_slider,
        retrain_button,
        retrain_output
    ], layout=widgets.Layout(gap='0.6em'))

    tabs = widgets.Tab(children=[tab1_box, tab2_box])
    tabs.set_title(0, 'Create Labeled Video')
    tabs.set_title(1, 'Labeling / Retraining')
    return tabs

prediction_tabs = build_prediction_workflow()
display(prediction_tabs)



### 📝 Napari Labeling Checklist

Follow these instructions while labeling in Napari after you click **Extract frames & launch Napari** above:

1. Open `Plugins → Keypoint controls` in Napari (dismiss the tutorial pop-up if it appears).
2. Choose `File → Open File(s)…`, navigate to the DeepLabCut project folder, and open `config.yaml`.
3. Choose `File → Open Folder…`, navigate into the new folder inside `labeled-data` that matches your video name, and select it. Then pick the `napari DeepLabCut` layer to start labeling.
4. After labeling at least five frames, save with `File → Save Selected Layer(s)` while `CollectedData` is selected, then close Napari before retraining.

Refer to the screenshots in the `assets/` folder (same images used in the Streamlit app) if you need a visual reminder.



## 📊 Step 2 — Post Processing

Use the interface below to reproduce the Streamlit **Post Processing** page. It is organised into three tabs:

1. **Labeled Data** — upload DLC prediction output, clean outliers, compute bending coefficients, and apply homography.
2. **Neuron Data** — upload neural recordings, inspect them, and downsample to match the video rate.
3. **Merged Data** — align both data sources, visualise receptive field maps, and export animations.

Run the cell below to load the widgets, then work through the tabs from left to right.


In [ ]:
# --- Post-processing interface ---

def build_labeled_data_tab() -> widgets.Accordion:
    style = {'description_width': '160px'}
    full_width = widgets.Layout(width='100%')

    dlc_path_text = widgets.Text(
        description='Existing path:',
        placeholder='Optional: /path/to/predictions.h5',
        style=style,
        layout=full_width
    )
    dlc_upload = widgets.FileUpload(
        accept='.h5',
        multiple=False,
        description='Upload .h5'
    )
    dlc_load_button = widgets.Button(description='Load DLC data', icon='upload')
    dlc_load_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})

    def on_load_dlc(_):
        with dlc_load_output:
            clear_output()
            path_value = dlc_path_text.value.strip()
            temp_path = None

            if path_value:
                candidate = Path(path_value.strip('"').strip("'"))
                if not candidate.exists():
                    print(f'File not found: {candidate}')
                    return
                temp_path = candidate
            elif dlc_upload.value:
                upload = next(iter(dlc_upload.value.values()))
                suffix = Path(upload['metadata']['name']).suffix or '.h5'
                temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=suffix)
                temp_file.write(upload['content'])
                temp_file.close()
                temp_path = Path(temp_file.name)
                NOTEBOOK_STATE['dlc_upload_path'] = temp_path
                print(f'Uploaded file stored at {temp_path}')
            else:
                print('Provide a path or upload a DLC prediction file (.h5).')
                return

            try:
                data_dlc = DataDLC(str(temp_path))
                NOTEBOOK_STATE['data_dlc'] = data_dlc
                NOTEBOOK_STATE['h5_path'] = temp_path
                NOTEBOOK_STATE['df_square_derivative_original'] = OutlierImputer.transform_to_derivative(data_dlc.df_square.copy())
                NOTEBOOK_STATE['df_monofil_derivative_original'] = OutlierImputer.transform_to_derivative(data_dlc.df_monofil.copy())
                NOTEBOOK_STATE['df_square_derivative_after'] = NOTEBOOK_STATE['df_square_derivative_original'].copy()
                NOTEBOOK_STATE['df_monofil_derivative_after'] = NOTEBOOK_STATE['df_monofil_derivative_original'].copy()

                display(Markdown('**DLC square points preview (first rows):**'))
                display(data_dlc.df_square.head())
                display(Markdown('**Average likelihoods:**'))
                likelihood_text = data_dlc.get_avg_likelihoods().replace('\n', '  \n')
                display(Markdown(likelihood_text))
            except Exception as exc:
                print(f'Failed to load DLC data: {exc}')

    dlc_load_button.on_click(on_load_dlc)

    imputation_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})
    std_square_input = widgets.FloatText(value=5.0, description='Square std', step=0.1, style=style)
    model_square_dropdown = widgets.Dropdown(
        options=['All Models'] + list(OutlierImputer.models.keys()),
        value='BR',
        description='Square model',
        style=style
    )
    square_impute_button = widgets.Button(description='Impute square outliers', icon='magic')

    std_filament_input = widgets.FloatText(value=5.0, description='Filament std', step=0.1, style=style)
    model_filament_dropdown = widgets.Dropdown(
        options=['All Models'] + list(OutlierImputer.models.keys()),
        value='BR',
        description='Filament model',
        style=style
    )
    filament_impute_button = widgets.Button(description='Impute filament outliers', icon='magic')

    def on_impute_square(_):
        with imputation_output:
            clear_output()
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if data_dlc is None:
                print('Load DLC data first.')
                return
            model = model_square_dropdown.value
            if model == 'All Models':
                model = None
            try:
                data_dlc.impute_outliers(
                    std_threshold=float(std_square_input.value),
                    square=True,
                    filament=False,
                    model_name=model
                )
                NOTEBOOK_STATE['df_square_derivative_after'] = OutlierImputer.transform_to_derivative(data_dlc.df_square.copy())
                print('Square points imputed successfully.')
                json_path = Path('latest_square.json')
                if json_path.exists():
                    with open(json_path, 'r') as f:
                        display(JSON(json.load(f)))
            except Exception as exc:
                print(f'Square imputation failed: {exc}')

    def on_impute_filament(_):
        with imputation_output:
            clear_output()
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if data_dlc is None:
                print('Load DLC data first.')
                return
            model = model_filament_dropdown.value
            if model == 'All Models':
                model = None
            try:
                data_dlc.impute_outliers(
                    std_threshold=float(std_filament_input.value),
                    square=False,
                    filament=True,
                    model_name=model
                )
                NOTEBOOK_STATE['df_monofil_derivative_after'] = OutlierImputer.transform_to_derivative(data_dlc.df_monofil.copy())
                print('Filament points imputed successfully.')
                json_path = Path('latest_filament.json')
                if json_path.exists():
                    with open(json_path, 'r') as f:
                        display(JSON(json.load(f)))
            except Exception as exc:
                print(f'Filament imputation failed: {exc}')

    square_impute_button.on_click(on_impute_square)
    filament_impute_button.on_click(on_impute_filament)

    derivative_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})
    plot_square_before_button = widgets.Button(description='Square derivative (before)', icon='line-chart')
    plot_square_after_button = widgets.Button(description='Square derivative (after)', icon='line-chart')
    plot_filament_before_button = widgets.Button(description='Filament derivative (before)', icon='line-chart')
    plot_filament_after_button = widgets.Button(description='Filament derivative (after)', icon='line-chart')

    def _plot_derivative(df: pd.DataFrame, title: str) -> None:
        fig = px.line(df, title=title)
        fig.update_layout(title_x=0.5, xaxis_title='Frame', yaxis_title='Derivative value')
        fig.show()

    def on_plot_square_before(_):
        with derivative_output:
            clear_output()
            df = NOTEBOOK_STATE.get('df_square_derivative_original')
            if df is None:
                print('Load DLC data first.')
                return
            _plot_derivative(df, 'Square derivative (before imputation)')

    def on_plot_square_after(_):
        with derivative_output:
            clear_output()
            df = NOTEBOOK_STATE.get('df_square_derivative_after')
            if df is None:
                print('Impute square outliers first.')
                return
            _plot_derivative(df, 'Square derivative (after imputation)')

    def on_plot_filament_before(_):
        with derivative_output:
            clear_output()
            df = NOTEBOOK_STATE.get('df_monofil_derivative_original')
            if df is None:
                print('Load DLC data first.')
                return
            _plot_derivative(df, 'Filament derivative (before imputation)')

    def on_plot_filament_after(_):
        with derivative_output:
            clear_output()
            df = NOTEBOOK_STATE.get('df_monofil_derivative_after')
            if df is None:
                print('Impute filament outliers first.')
                return
            _plot_derivative(df, 'Filament derivative (after imputation)')

    plot_square_before_button.on_click(on_plot_square_before)
    plot_square_after_button.on_click(on_plot_square_after)
    plot_filament_before_button.on_click(on_plot_filament_before)
    plot_filament_after_button.on_click(on_plot_filament_after)

    labeled_video_path_input = widgets.Text(
        value=str(NOTEBOOK_STATE.get('processed_video_path', '')),
        description='Video path:',
        style=style,
        layout=full_width
    )
    labeled_video_button = widgets.Button(description='Generate labeled video', icon='video')
    labeled_video_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})

    def on_generate_labeled_video(_):
        with labeled_video_output:
            clear_output()
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if data_dlc is None:
                print('Load DLC data first.')
                return
            video_path_value = labeled_video_path_input.value.strip()
            if not video_path_value:
                print('Provide the path to the source video (preprocessed).')
                return
            video_path = Path(video_path_value.strip('"').strip("'"))
            if not video_path.exists():
                print(f'Video not found: {video_path}')
                return
            try:
                video_bytes = PlottingPlotly.generate_labeled_video(data_dlc, str(video_path))
                display(Video(data=video_bytes, embed=True))
                NOTEBOOK_STATE['labeled_video_path'] = video_path
            except Exception as exc:
                print(f'Failed to generate labeled video: {exc}')

    labeled_video_button.on_click(on_generate_labeled_video)

    bending_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})
    compute_bending_button = widgets.Button(description='Compute bending coefficients', icon='chart-line')
    bending_plot_button = widgets.Button(description='Plot bending coefficients', icon='line-chart')
    bending_title = widgets.Text(value='Bending Coefficients', description='Title', style=style)
    bending_xlabel = widgets.Text(value='Frame', description='X axis', style=style)
    bending_ylabel = widgets.Text(value='Bending value', description='Y axis', style=style)
    bending_color_picker = widgets.ColorPicker(value='#00f900', description='Line colour')

    def on_compute_bending(_):
        with bending_output:
            clear_output()
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if data_dlc is None:
                print('Load DLC data first.')
                return
            try:
                data_dlc.get_bending_coefficients()
                NOTEBOOK_STATE['df_bending_coefficients'] = data_dlc.df_bending_coefficients
                print('Bending coefficients calculated. Preview:')
                display(data_dlc.df_bending_coefficients.head())
            except Exception as exc:
                print(f'Failed to compute bending coefficients: {exc}')

    def on_plot_bending(_):
        with bending_output:
            clear_output()
            data = NOTEBOOK_STATE.get('df_bending_coefficients')
            if data is None:
                print('Compute bending coefficients first.')
                return
            df = pd.DataFrame({'Frame': range(len(data)), 'Bending': data})
            fig = px.line(
                df,
                x='Frame',
                y='Bending',
                title=bending_title.value,
                color_discrete_sequence=[bending_color_picker.value]
            )
            fig.update_layout(title_x=0.5, xaxis_title=bending_xlabel.value, yaxis_title=bending_ylabel.value)
            fig.show()

    compute_bending_button.on_click(on_compute_bending)
    bending_plot_button.on_click(on_plot_bending)

    homography_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})
    homography_video_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})
    homo_min_input = widgets.IntText(value=0, description='Minimum', style=style)
    homo_max_input = widgets.IntText(value=20, description='Maximum', style=style)
    apply_homography_button = widgets.Button(description='Apply homography', icon='project-diagram')
    homography_plot_button = widgets.Button(description='Interactive plot', icon='map')
    homography_video_button = widgets.Button(description='Homography animation', icon='film')
    homography_fig_width = widgets.IntText(value=12, description='Figure width', style=style)
    homography_fig_height = widgets.IntText(value=12, description='Figure height', style=style)
    homography_video_fps = widgets.IntText(value=30, description='FPS', style=style)

    def on_apply_homography(_):
        with homography_output:
            clear_output()
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if data_dlc is None:
                print('Load DLC data first.')
                return
            try:
                data_dlc.assign_homography_points(int(homo_min_input.value), int(homo_max_input.value))
                data_dlc.apply_homography()
                NOTEBOOK_STATE['homography_applied'] = True
                print('Homography applied. Transformed monofilament preview:')
                display(data_dlc.df_transformed_monofil.head())
            except Exception as exc:
                print(f'Failed to apply homography: {exc}')

    def on_plot_homography(_):
        with homography_output:
            clear_output()
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if data_dlc is None or data_dlc.homography_points is None:
                print('Apply homography first.')
                return
            try:
                fig = PlottingPlotly.plot_homography_interactive(
                    homography_points=data_dlc.homography_points,
                    df_transformed_monofil=data_dlc.df_transformed_monofil,
                    title='Homography Plot',
                    x_label='x (mm)',
                    y_label='y (mm)',
                    color='#00f900'
                )
                fig.show()
            except Exception as exc:
                print(f'Failed to plot homography: {exc}')

    def on_homography_video(_):
        with homography_video_output:
            clear_output()
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if data_dlc is None or data_dlc.homography_points is None:
                print('Apply homography first.')
                return
            try:
                figsize = (int(homography_fig_width.value), int(homography_fig_height.value))
                video_bytes = PlottingPlotly.generate_homography_video(
                    data_dlc.homography_points,
                    data_dlc.df_transformed_monofil,
                    fps=int(homography_video_fps.value),
                    title='Homography Animation',
                    x_label='x (mm)',
                    y_label='y (mm)',
                    color='#00f900',
                    figsize=figsize
                )
                display(Video(data=video_bytes, embed=True))
            except Exception as exc:
                print(f'Failed to generate homography video: {exc}')

    apply_homography_button.on_click(on_apply_homography)
    homography_plot_button.on_click(on_plot_homography)
    homography_video_button.on_click(on_homography_video)

    upload_box = widgets.VBox([
        widgets.HTML('<h4>Load DLC data</h4>'),
        dlc_path_text,
        dlc_upload,
        dlc_load_button,
        dlc_load_output
    ], layout=widgets.Layout(gap='0.4em'))

    imputation_box = widgets.VBox([
        widgets.HTML('<h4>Outlier imputation</h4>'),
        widgets.HBox([std_square_input, model_square_dropdown, square_impute_button]),
        widgets.HBox([std_filament_input, model_filament_dropdown, filament_impute_button]),
        imputation_output
    ], layout=widgets.Layout(gap='0.4em'))

    derivative_box = widgets.VBox([
        widgets.HTML('<h4>Derivative plots</h4>'),
        widgets.HBox([plot_square_before_button, plot_square_after_button]),
        widgets.HBox([plot_filament_before_button, plot_filament_after_button]),
        derivative_output
    ], layout=widgets.Layout(gap='0.4em'))

    labeled_video_box = widgets.VBox([
        widgets.HTML('<h4>Regenerate labeled video</h4>'),
        labeled_video_path_input,
        labeled_video_button,
        labeled_video_output
    ], layout=widgets.Layout(gap='0.4em'))

    bending_box = widgets.VBox([
        widgets.HTML('<h4>Bending coefficients</h4>'),
        compute_bending_button,
        widgets.HBox([bending_plot_button, bending_color_picker]),
        widgets.HBox([bending_title, bending_xlabel, bending_ylabel]),
        bending_output
    ], layout=widgets.Layout(gap='0.4em'))

    homography_box = widgets.VBox([
        widgets.HTML('<h4>Homography & animation</h4>'),
        widgets.HBox([homo_min_input, homo_max_input, apply_homography_button]),
        widgets.HBox([homography_plot_button, homography_video_button]),
        widgets.HBox([homography_fig_width, homography_fig_height, homography_video_fps]),
        homography_output,
        homography_video_output
    ], layout=widgets.Layout(gap='0.4em'))

    accordion = widgets.Accordion(children=[
        upload_box,
        imputation_box,
        derivative_box,
        labeled_video_box,
        bending_box,
        homography_box
    ])
    titles = [
        'Load DLC data',
        'Outlier imputation',
        'Derivative plots',
        'Regenerate labeled video',
        'Bending coefficients',
        'Homography'
    ]
    for idx, title in enumerate(titles):
        accordion.set_title(idx, title)
    return accordion


def build_neuron_data_tab() -> widgets.Accordion:
    style = {'description_width': '160px'}
    full_width = widgets.Layout(width='100%')

    neuron_path_text = widgets.Text(
        description='Existing path:',
        placeholder='Optional: /path/to/neuron.csv or .xlsx',
        style=style,
        layout=full_width
    )
    neuron_upload = widgets.FileUpload(
        accept='.csv,.xlsx',
        multiple=False,
        description='Upload data'
    )
    original_fps_input = widgets.IntText(value=0, description='Original sample rate', style=style)
    target_fps_input = widgets.IntText(value=30, description='Target sample rate', style=style)

    neuron_load_button = widgets.Button(description='Load neuron data', icon='upload')
    neuron_load_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})

    def on_load_neuron(_):
        with neuron_load_output:
            clear_output()
            path_value = neuron_path_text.value.strip()
            temp_path = None
            if path_value:
                candidate = Path(path_value.strip('"').strip("'"))
                if not candidate.exists():
                    print(f'File not found: {candidate}')
                    return
                temp_path = candidate
            elif neuron_upload.value:
                upload = next(iter(neuron_upload.value.values()))
                suffix = Path(upload['metadata']['name']).suffix or '.csv'
                temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=suffix)
                temp_file.write(upload['content'])
                temp_file.close()
                temp_path = Path(temp_file.name)
                NOTEBOOK_STATE['neuron_upload_path'] = temp_path
                print(f'Uploaded file stored at {temp_path}')
            else:
                print('Provide a path or upload a neuron data file (.csv or .xlsx).')
                return

            if not original_fps_input.value or original_fps_input.value <= 0:
                print('Set the original sample rate (Hz) before loading.')
                return

            try:
                data_neuron = DataNeuron(str(temp_path), int(original_fps_input.value))
                NOTEBOOK_STATE['neuron_data'] = data_neuron
                NOTEBOOK_STATE['neuron_path'] = temp_path
                display(Markdown('**Neuron data preview:**'))
                display(data_neuron.df.head())
            except Exception as exc:
                print(f'Failed to load neuron data: {exc}')

    neuron_load_button.on_click(on_load_neuron)

    neuron_plot_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})
    neuron_plot_button = widgets.Button(description='Plot neuron data', icon='area-chart')
    neuron_plot_title = widgets.Text(value='Neuron data, original sample rate', description='Title', style=style)
    neuron_plot_xlabel = widgets.Text(value='Index', description='X axis', style=style)
    neuron_plot_ylabel1 = widgets.Text(value='Neuron spikes', description='Y1 axis', style=style)
    neuron_plot_ylabel2 = widgets.Text(value='IFF', description='Y2 axis', style=style)
    neuron_color1 = widgets.ColorPicker(value='#1f77b4', description='Spike colour')
    neuron_color2 = widgets.ColorPicker(value='#d62728', description='IFF colour')
    neuron_invert_checkbox = widgets.Checkbox(value=False, description='Invert IFF axis')

    def on_plot_neuron(_):
        with neuron_plot_output:
            clear_output()
            data_neuron: DataNeuron | None = NOTEBOOK_STATE.get('neuron_data')
            if data_neuron is None:
                print('Load neuron data first.')
                return
            try:
                fig = PlottingPlotly.plot_dual_y_axis(
                    df=data_neuron.df,
                    columns=['Spike', 'IFF'],
                    xlabel=neuron_plot_xlabel.value,
                    ylabel_1=neuron_plot_ylabel1.value,
                    ylabel_2=neuron_plot_ylabel2.value,
                    title=neuron_plot_title.value,
                    color_1=neuron_color1.value,
                    color_2=neuron_color2.value,
                    invert_y_2=neuron_invert_checkbox.value
                )
                fig.show()
            except Exception as exc:
                print(f'Failed to plot neuron data: {exc}')

    neuron_plot_button.on_click(on_plot_neuron)

    neuron_downsample_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})
    neuron_downsample_button = widgets.Button(description='Downsample neuron data', icon='compress')
    neuron_downsample_plot_button = widgets.Button(description='Plot downsampled data', icon='area-chart')

    def on_downsample_neuron(_):
        with neuron_downsample_output:
            clear_output()
            data_neuron: DataNeuron | None = NOTEBOOK_STATE.get('neuron_data')
            if data_neuron is None:
                print('Load neuron data first.')
                return
            if not target_fps_input.value or target_fps_input.value <= 0:
                print('Set the target sample rate (Hz) before downsampling.')
                return
            try:
                data_neuron.downsample(int(target_fps_input.value))
                NOTEBOOK_STATE['neuron_downsampled_df'] = data_neuron.downsampled_df
                print('Downsampled data preview:')
                display(data_neuron.downsampled_df.head())
            except Exception as exc:
                print(f'Downsampling failed: {exc}')

    def on_plot_downsampled(_):
        with neuron_downsample_output:
            clear_output()
            data_neuron: DataNeuron | None = NOTEBOOK_STATE.get('neuron_data')
            if data_neuron is None or data_neuron.downsampled_df is None:
                print('Downsample the neuron data first.')
                return
            try:
                fig = PlottingPlotly.plot_dual_y_axis(
                    df=data_neuron.downsampled_df,
                    columns=['Spike', 'IFF'],
                    xlabel='Frame',
                    ylabel_1='Sum of spikes',
                    ylabel_2='Max IFF',
                    title='Neuron data (downsampled)',
                    color_1=neuron_color1.value,
                    color_2=neuron_color2.value,
                    invert_y_2=neuron_invert_checkbox.value
                )
                fig.show()
            except Exception as exc:
                print(f'Failed to plot downsampled data: {exc}')

    neuron_downsample_button.on_click(on_downsample_neuron)
    neuron_downsample_plot_button.on_click(on_plot_downsampled)

    load_box = widgets.VBox([
        widgets.HTML('<h4>Load neuron data</h4>'),
        neuron_path_text,
        neuron_upload,
        widgets.HBox([original_fps_input, target_fps_input, neuron_load_button]),
        neuron_load_output
    ], layout=widgets.Layout(gap='0.4em'))

    plot_box = widgets.VBox([
        widgets.HTML('<h4>Visualise original data</h4>'),
        widgets.HBox([neuron_plot_button, neuron_invert_checkbox]),
        widgets.HBox([neuron_plot_title, neuron_plot_xlabel]),
        widgets.HBox([neuron_plot_ylabel1, neuron_plot_ylabel2]),
        widgets.HBox([neuron_color1, neuron_color2]),
        neuron_plot_output
    ], layout=widgets.Layout(gap='0.4em'))

    downsample_box = widgets.VBox([
        widgets.HTML('<h4>Downsample & inspect</h4>'),
        widgets.HBox([neuron_downsample_button, neuron_downsample_plot_button]),
        neuron_downsample_output
    ], layout=widgets.Layout(gap='0.4em'))

    accordion = widgets.Accordion(children=[load_box, plot_box, downsample_box])
    accordion.set_title(0, 'Load neuron data')
    accordion.set_title(1, 'Visualise original data')
    accordion.set_title(2, 'Downsample & inspect')
    return accordion


def build_merged_data_tab() -> widgets.Accordion:
    style = {'description_width': '160px'}

    max_gap_fill_slider = widgets.IntSlider(value=10, min=1, max=50, step=1, description='Max gap fill', style=style)
    threshold_slider = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, step=0.05, description='Z-score threshold', style=style)
    merge_button = widgets.Button(description='Merge DLC & neuron data', icon='link')
    merge_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})

    def refresh_column_options():
        merged = NOTEBOOK_STATE.get('merged_data')
        if merged is None:
            return
        columns = merged.df_merged.columns.tolist()
        size_dropdown.options = columns
        color_dropdown.options = columns
        if 'Bending_ZScore' in columns:
            size_dropdown.value = 'Bending_ZScore'
        if 'Spike' in columns:
            color_dropdown.value = 'Spike'

    def on_merge_clicked(_):
        with merge_output:
            clear_output()
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            neuron_data: DataNeuron | None = NOTEBOOK_STATE.get('neuron_data')
            if data_dlc is None or neuron_data is None:
                print('Load DLC and neuron data before merging.')
                return
            try:
                merged = MergedData(
                    data_dlc,
                    neuron_data,
                    max_gap_fill=int(max_gap_fill_slider.value),
                    threshold=float(threshold_slider.value)
                )
                NOTEBOOK_STATE['merged_data'] = merged
                print('Merged data preview:')
                display(merged.df_merged.head())
                refresh_column_options()
            except Exception as exc:
                print(f'Merging failed: {exc}')

    merge_button.on_click(on_merge_clicked)

    bending_toggle = widgets.Checkbox(value=True, description='Apply bending threshold')
    spikes_toggle = widgets.Checkbox(value=True, description='Require spikes')
    filter_button = widgets.Button(description='Preview filtered rows', icon='table')
    filter_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})

    def on_filter_clicked(_):
        with filter_output:
            clear_output()
            merged: MergedData | None = NOTEBOOK_STATE.get('merged_data')
            if merged is None:
                print('Merge the data first.')
                return
            df = merged.threshold_data(bending=bending_toggle.value, spikes=spikes_toggle.value)
            print(f'Rows after filtering: {len(df)}')
            display(df.head())

    filter_button.on_click(on_filter_clicked)

    plot_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})
    kde_button = widgets.Button(description='Interactive KDE', icon='satellite')
    scatter_button = widgets.Button(description='Interactive scatter', icon='braille')
    plot_title_input = widgets.Text(value='KDE / Scatter Plot', description='Title', style=style)
    plot_xlabel_input = widgets.Text(value='x (mm)', description='X axis', style=style)
    plot_ylabel_input = widgets.Text(value='y (mm)', description='Y axis', style=style)
    size_dropdown = widgets.Dropdown(description='Size column')
    color_dropdown = widgets.Dropdown(description='Colour column')
    plotly_cmap_dropdown = widgets.Dropdown(options=sorted(PLOTLY_CMAPS.keys()), value='Viridis', description='Plotly cmap')
    plotly_spikes_cmap_dropdown = widgets.Dropdown(options=sorted(PLOTLY_CMAPS.keys()), value='Inferno', description='Spikes cmap')
    bw_bending_input = widgets.FloatText(value=0.1, description='Bandwidth (bending)', style=style)
    bw_spikes_input = widgets.FloatText(value=0.1, description='Bandwidth (spikes)', style=style)
    bw_threshold_input = widgets.FloatText(value=0.05, description='% threshold', style=style)

    def on_kde_clicked(_):
        with plot_output:
            clear_output()
            merged: MergedData | None = NOTEBOOK_STATE.get('merged_data')
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if merged is None or data_dlc is None or data_dlc.homography_points is None:
                print('Merge data and apply homography before plotting.')
                return
            try:
                fig = PlottingPlotly.plot_kde_density_interactive(
                    merged,
                    x_col='tf_FB2_x',
                    y_col='tf_FB2_y',
                    homography_points=data_dlc.homography_points,
                    bending=bending_toggle.value,
                    spikes=spikes_toggle.value,
                    title=plot_title_input.value,
                    xlabel=plot_xlabel_input.value,
                    ylabel=plot_ylabel_input.value,
                    cmap_bending=plotly_cmap_dropdown.value,
                    cmap_spikes=plotly_spikes_cmap_dropdown.value,
                    bw_bending=float(bw_bending_input.value),
                    bw_spikes=float(bw_spikes_input.value),
                    threshold_percentage=float(bw_threshold_input.value)
                )
                fig.show()
            except Exception as exc:
                print(f'Failed to plot KDE: {exc}')

    def on_scatter_clicked(_):
        with plot_output:
            clear_output()
            merged: MergedData | None = NOTEBOOK_STATE.get('merged_data')
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if merged is None or data_dlc is None or data_dlc.homography_points is None:
                print('Merge data and apply homography before plotting.')
                return
            try:
                fig = PlottingPlotly.plot_scatter_interactive(
                    merged,
                    x_col='tf_FB2_x',
                    y_col='tf_FB2_y',
                    homography_points=data_dlc.homography_points,
                    size_col=size_dropdown.value,
                    color_col=color_dropdown.value,
                    bending=bending_toggle.value,
                    spikes=spikes_toggle.value,
                    title=plot_title_input.value,
                    xlabel=plot_xlabel_input.value,
                    ylabel=plot_ylabel_input.value,
                    cmap=plotly_cmap_dropdown.value
                )
                fig.show()
            except Exception as exc:
                print(f'Failed to plot scatter: {exc}')

    kde_button.on_click(on_kde_clicked)
    scatter_button.on_click(on_scatter_clicked)

    animation_output = widgets.Output(layout={'border': '1px solid #ddd', 'padding': '0.2em'})
    rf_video_button = widgets.Button(description='RF map animation (video)', icon='play')
    scatter_video_button = widgets.Button(description='Scatter animation (video)', icon='play-circle')
    scroll_video_button = widgets.Button(description='Scrolling overlay video', icon='video-camera')
    animation_fps_input = widgets.IntText(value=30, description='Video FPS', style=style)
    animation_fig_width = widgets.IntText(value=12, description='Figure width', style=style)
    animation_fig_height = widgets.IntText(value=12, description='Figure height', style=style)

    def on_rf_video(_):
        with animation_output:
            clear_output()
            merged: MergedData | None = NOTEBOOK_STATE.get('merged_data')
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if merged is None or data_dlc is None or data_dlc.homography_points is None:
                print('Merge data and apply homography before generating the animation.')
                return
            try:
                figsize = (int(animation_fig_width.value), int(animation_fig_height.value))
                video_bytes = PlottingPlotly.plot_rf_mapping_animated(
                    merged,
                    x_col='tf_FB2_x',
                    y_col='tf_FB2_y',
                    homography_points=data_dlc.homography_points,
                    size_col=size_dropdown.value,
                    color_col=color_dropdown.value,
                    title=plot_title_input.value,
                    bending=bending_toggle.value,
                    spikes=spikes_toggle.value,
                    xlabel=plot_xlabel_input.value,
                    ylabel=plot_ylabel_input.value,
                    fps=int(animation_fps_input.value),
                    figsize=figsize,
                    cmap=plotly_cmap_dropdown.value
                )
                display(Video(data=video_bytes, embed=True))
            except Exception as exc:
                print(f'RF map animation failed: {exc}')

    def on_scatter_video(_):
        with animation_output:
            clear_output()
            merged: MergedData | None = NOTEBOOK_STATE.get('merged_data')
            data_dlc: DataDLC | None = NOTEBOOK_STATE.get('data_dlc')
            if merged is None or data_dlc is None or data_dlc.homography_points is None:
                print('Merge data and apply homography before generating the animation.')
                return
            if not hasattr(PlottingPlotly, 'generate_scatter_plot_animation'):
                print('Scatter animation helper is not available in this repository version.')
                return
            try:
                figsize = (int(animation_fig_width.value), int(animation_fig_height.value))
                video_bytes = PlottingPlotly.generate_scatter_plot_animation(
                    merged,
                    x_col='tf_FB2_x',
                    y_col='tf_FB2_y',
                    homography_points=data_dlc.homography_points,
                    size_col=size_dropdown.value,
                    color_col=color_dropdown.value,
                    bending=bending_toggle.value,
                    spikes=spikes_toggle.value,
                    title=plot_title_input.value,
                    xlabel=plot_xlabel_input.value,
                    ylabel=plot_ylabel_input.value,
                    cmap=plotly_cmap_dropdown.value,
                    fps=int(animation_fps_input.value),
                    figsize=figsize
                )
                display(Video(data=video_bytes, embed=True))
            except Exception as exc:
                print(f'Scatter animation failed: {exc}')

    def on_scroll_video(_):
        with animation_output:
            clear_output()
            merged: MergedData | None = NOTEBOOK_STATE.get('merged_data')
            video_path = NOTEBOOK_STATE.get('labeled_video_path')
            if merged is None:
                print('Merge data first.')
                return
            if video_path is None or not Path(video_path).exists():
                print('Provide a labeled video path in the DLC tab before generating the scrolling overlay video.')
                return
            try:
                video_bytes = PlottingPlotly.generate_scroll_over_video(
                    merged_data=merged,
                    columns=['Bending_ZScore', 'Spike'],
                    video_path=str(video_path),
                    color_1='#1f77b4',
                    color_2='#d62728',
                    title='Scrolling Overlay Video'
                )
                display(Video(data=video_bytes, embed=True))
            except Exception as exc:
                print(f'Scrolling overlay failed: {exc}')

    rf_video_button.on_click(on_rf_video)
    scatter_video_button.on_click(on_scatter_video)
    scroll_video_button.on_click(on_scroll_video)

    merge_box = widgets.VBox([
        widgets.HTML('<h4>Merge DLC & neuron data</h4>'),
        widgets.HBox([max_gap_fill_slider, threshold_slider, merge_button]),
        merge_output
    ], layout=widgets.Layout(gap='0.4em'))

    filter_box = widgets.VBox([
        widgets.HTML('<h4>Filter preview</h4>'),
        widgets.HBox([bending_toggle, spikes_toggle, filter_button]),
        filter_output
    ], layout=widgets.Layout(gap='0.4em'))

    plot_box = widgets.VBox([
        widgets.HTML('<h4>Interactive plots</h4>'),
        widgets.HBox([kde_button, scatter_button]),
        widgets.HBox([plot_title_input, plot_xlabel_input, plot_ylabel_input]),
        size_dropdown,
        color_dropdown,
        plotly_cmap_dropdown,
        plotly_spikes_cmap_dropdown,
        widgets.HBox([bw_bending_input, bw_spikes_input, bw_threshold_input]),
        plot_output
    ], layout=widgets.Layout(gap='0.4em'))

    animation_box = widgets.VBox([
        widgets.HTML('<h4>Animations & videos</h4>'),
        widgets.HBox([rf_video_button, scatter_video_button, scroll_video_button]),
        widgets.HBox([animation_fps_input, animation_fig_width, animation_fig_height]),
        animation_output
    ], layout=widgets.Layout(gap='0.4em'))

    accordion = widgets.Accordion(children=[merge_box, filter_box, plot_box, animation_box])
    accordion.set_title(0, 'Merge data')
    accordion.set_title(1, 'Filter preview')
    accordion.set_title(2, 'Interactive plots')
    accordion.set_title(3, 'Animations & videos')
    return accordion


post_processing_tabs = widgets.Tab(children=[
    build_labeled_data_tab(),
    build_neuron_data_tab(),
    build_merged_data_tab()
])
post_processing_tabs.set_title(0, 'Labeled Data')
post_processing_tabs.set_title(1, 'Neuron Data')
post_processing_tabs.set_title(2, 'Merged Data')

display(post_processing_tabs)



## ✅ Next Steps

- Step through the tabs from left to right to mirror the Streamlit workflow.
- Re-run individual widget sections whenever you adjust parameters; results will update live.
- Generated videos can be saved by right-clicking in the output or by adapting the helper functions to write to disk.
- Once satisfied, proceed with your downstream analysis or export the processed dataframes for further work.
